In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import io
import xml.etree.ElementTree as ET

In [2]:
Global_df = pd.read_excel('CNMV_Global_Scrape.xlsx')
Global_df

,Type,AM Name,AM Code,Global Data URL,Fund Name,Fund Code,Fund Global URL,ID Fund
0,IIC,"360 CORA SGIIC, S.A.",276,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...,"CODEX GLOBAL FUND, FI",5461.0,https://www.cnmv.es/Portal/Consultas/IIC/Fondo...,276-5461
1,IIC,"A&G FONDOS, SGIIC, SA",195,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...,"A&G RENTA FIJA CORTO PLAZO, FI",3989.0,https://www.cnmv.es/Portal/Consultas/IIC/Fondo...,195-3989
2,IIC,"A&G FONDOS, SGIIC, SA",195,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...,"GLOBAL MANAGERS FUNDS, FI",3072.0,https://www.cnmv.es/Portal/Consultas/IIC/Fondo...,195-3072
3,IIC,"A&G FONDOS, SGIIC, SA",195,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...,"GREDOS BOLSA EURO, FI",4881.0,https://www.cnmv.es/Portal/Consultas/IIC/Fondo...,195-4881
4,IIC,"A&G FONDOS, SGIIC, SA",195,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...,"GREDOS BOLSA INTERNACIONAL, FI",4883.0,https://www.cnmv.es/Portal/Consultas/IIC/Fondo...,195-4883
5,IIC,"A&G FONDOS, SGIIC, SA",195,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...,"GREDOS MODERADO, FI",4882.0,https://www.cnmv.es/Portal/Consultas/IIC/Fondo...,195-4882
6,IIC,"A&G FONDOS, SGIIC, SA",195,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...,"GREDOS RENTA FIJA, FI",5389.0,https://www.cnmv.es/Portal/Consultas/IIC/Fondo...,195-5389
7,IIC,"ABACO CAPITAL, SGIIC, S.A.",238,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...,ABACO GLOBAL VALUE OPPORTUNITIES FI,4827.0,https://www.cnmv.es/Portal/Consultas/IIC/Fondo...,238-4827
8,IIC,"ABACO CAPITAL, SGIIC, S.A.",238,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...,"ABACO RENTA FIJA MIXTA GLOBAL, FI",4474.0,https://www.cnmv.es/Portal/Consultas/IIC/Fondo...,238-4474
9,IIC,"ABANTE ASESORES GESTION, SGIIC, S.A.",194,https://www.cnmv.es/Portal/Consultas/IIC/SGIIC...,"ABANTE ASESORES GLOBAL, FI",2562.0,https://www.cnmv.es/Portal/Consultas/IIC/Fondo...,194-2562


In [3]:
Global_df.iloc[0]['Fund Global URL']

'https://www.cnmv.es/Portal/Consultas/IIC/Fondo.aspx?nif=V88644323 '

In [5]:
#Convert xml format to Beautifulsoup
content = []
# Read the XML file
with io.open('Info Pública/2-1294.xml', 'r', encoding="utf-8") as file:
    # Read each line in the file, readlines() returns a list of lines
    content = file.readlines()
    # Combine the lines in the list into a string
    content = "".join(content)
    bs_content = BeautifulSoup(content, "xml")
    
#contextRef is an attribute of each file
#adding '_ia' makes reference to current value and '_ipp' makes reference to past value   
contextRef = bs_content.find('context')['id'].replace('_da','')

In [6]:
#Scrape the portfolio information
port_soup = bs_content.find_all(['CodigoISIN',
                                 'InversionesFinancierasDescripcion',
                                 'InversionesFinancierasPorcentaje'], {'contextRef': [contextRef + '_ia', contextRef + '_ipp']})

list_ISIN = []
list_asset = []
list_description = []
list_weights_curr = []
list_weights_past = []

for i in range(len(port_soup)):
    if port_soup[i].name == 'CodigoISIN' and port_soup[i+1].name == 'InversionesFinancierasDescripcion' and port_soup[i+2].name == 'InversionesFinancierasPorcentaje' and port_soup[i+3].name == 'InversionesFinancierasPorcentaje':
        
        list_ISIN += [port_soup[i].text]
        i_aux_des = port_soup[i+1].text
        list_asset += [i_aux_des.split('|')[0]]
        list_description += [i_aux_des]
        list_weights_curr += [float(port_soup[i+2].text)/100]
        list_weights_past += [float(port_soup[i+3].text)/100]
        
portfolio = pd.DataFrame(data={'ISIN': list_ISIN, 'Activo': list_asset, 'Descripción': list_description,'Peso Actual': list_weights_curr, 'Peso Anterior': list_weights_past})

other_weights = 1 - portfolio[['Peso Actual', 'Peso Anterior']].sum()
portfolio = portfolio.append(pd.DataFrame(data={'ISIN': np.nan, 'Activo': 'Tesorería y Otros', 'Descripción': 'Tesorería y Otros', 'Peso Actual': other_weights[0], 'Peso Anterior': other_weights[1]}, index=[0]), ignore_index=True)
portfolio

,ISIN,Activo,Descripción,Peso Actual,Peso Anterior
0,USP3579ECH82,OBLIGACION,"OBLIGACION|REPUBLICA DOMINICANA|4,88|2032-09-23",0.0043,0.0000
1,XS2080771806,OBLIGACION,"OBLIGACION|DEUDA ESTADO MARRUEC|1,50|2031-11-27",0.0128,0.0000
2,XS2135361686,OBLIGACION,"OBLIGACION|DEUDA ESTADO MEXICAN|1,35|2027-06-18",0.0025,0.0000
3,XS2214239506,OBLIGACION,"OBLIGACION|REPUBLICA ECUADOR|13,35|2030-07-31",0.0006,0.0000
4,XS2214239175,OBLIGACION,"OBLIGACION|REPUBLICA ECUADOR|0,50|2040-07-31",0.0122,0.0000
5,XS2214238441,OBLIGACION,"OBLIGACION|REPUBLICA ECUADOR|0,50|2035-07-31",0.0113,0.0000
6,XS2214237807,OBLIGACION,"OBLIGACION|REPUBLICA ECUADOR|0,50|2030-07-31",0.0042,0.0000
7,US91282CAE12,OBLIGACION,"OBLIGACION|DEUDA ESTADO USA|0,63|2030-08-15",0.0218,0.0000
8,XS2120091991,OBLIGACION,"OBLIGACION|REPUBLICA BIELORRUSI|5,88|2026-02-24",0.0060,0.0000
9,XS1821416408,OBLIGACION,"OBLIGACION|REPUBLICA GHANA|8,63|2049-06-16",0.0080,0.0040
